# spinodal model

## stlfile

In [ ]:
import src.agent as agent
import numpy as np
import matplotlib.pyplot as plt
import pyacvd
import pyvista as pv
import numpy as np

grid = pv.ImageData(
    dimensions=(201,201,201),
    spacing=(0.5,0.5,0.5),
)

import os
dirc = 'spinodal_model'
os.system(f'mkdir -p {dirc}')
size = 100
theta = np.linspace(0,1,6) 
le= 5

for i in range(5):
    for j in range(5-i):
        for k in range(5-i-j):
            t_1 = theta[i]
            t_2 = theta[j]
            t_3 = theta[k]
            if t_1 != 0 or t_2 !=0 or t_3 !=0:
                for m in [5,7,10]:
                    spin = agent.Spinodal(a=1,b=1,c=1,t_1=t_1,t_2=t_2,t_3=t_3,l=1000,beta=m*np.pi/size)
                    print(t_1,t_2,t_3,spin.N.shape)
                    values = np.array([spin(X) for X in grid.points])
                    surf = grid.contour([0],scalars=values)
                    mesh = surf.triangulate(progress_bar=True)
                    M = pyacvd.Clustering(mesh)
                    # M.plot()
                    M.cluster(20000)
                    # M.plot()
                    # M.create_mesh().plot()
                    mesh = M.create_mesh()
                    print(f'{t_1}_{t_2}_{t_3}')
                    pv.save_meshio(dirc+f'/{t_1}_{t_2}_{t_3}_{m}.stl',M.create_mesh())


## to in file 

In [ ]:
import src.agent as agent
import numpy as np
import matplotlib.pyplot as plt
import pyacvd
import pyvista as pv
import numpy as np

m = 10
size = 100
grid = pv.UniformGrid(
    dimensions=(201,201,201),
    spacing=(0.5,0.5,0.5),
)


spin = agent.Spinodal(a=1,b=1,c=1,t_1=0.2,t_2=0.2,t_3=0.2,beta=m*np.pi/size)


values = np.array([spin(X) for X in grid.points])
surf = grid.contour([0],scalars=values)
mesh = surf.triangulate(progress_bar=True)
# M = pyacvd.Clustering(mesh)
# # M.plot()
# M.cluster(20000)

pv.save_meshio(f'{m}.stl',mesh)

In [ ]:
mesh = M.create_mesh()
points = mesh.points

import pyvista as pv
cloud = pv.PolyData(points)
surf = cloud.delaunay_2d(alpha=0.1)
surf.plot()

# Test

In [ ]:
import src.agent as agent
import numpy as np
import matplotlib.pyplot as plt


size = 100
spin = agent.Spinodal(a=1,b=1,c=1,t_1=0.2,t_2=0.2,t_3=0.2,beta=10*np.pi/100)

import pyvista as pv
import numpy as np

grid = pv.UniformGrid(
    dims=(201,201,201),
    spacing=(0.5,0.5,0.5),
)


# grid.points
values = np.array([spin(X) for X in grid.points])

In [ ]:

surf = grid.contour([0],scalars=values)
mesh = surf.triangulate(progress_bar=True)


# import pyacvd
# M = pyacvd.Clustering(mesh)
# M.plot()
# surf.plot()

In [ ]:
import pyacvd
# mesh = pv.read('spinodal.stl')
# surf_poly=pv.PolyData(mesh.vertices,mesh.faces)
M = pyacvd.Clustering(mesh)
# M.plot()
M.cluster(20000)
M.plot()
M.create_mesh().plot()
pv.save_meshio('spinodal.stl',M.create_mesh())

# build MD

## test

In [ ]:
import src.agent as agent
import src.stl2in as stl2in
import numpy as np
from numba import njit,jit

file = 'spinodal_model/0.2_0.2_0.2_10.stl.stl'

size = np.array([[0,100],[0,100],[0,100]])

import pyvista as pv
mesh  = pv.read(file)

def process_mesh(mesh):
    # Extract points from the mesh
    points = mesh.points

    # Prepare faces
    faces = np.array(mesh.faces, dtype=np.uint32).reshape((-1, 4))[:, 1:]
    neighbour_info = []
    bonds = []

    # Loop through each face in the mesh
    for face in faces:
        # Create bond pairs for each unique pair in the face
        for vertex1 in face:
            for vertex2 in face:
                if vertex1 < vertex2:
                    bonds.append([vertex1, vertex2])

    # Remove duplicate bond pairs
    bonds = np.unique(np.array(bonds, dtype=np.uint32), axis=0)
    bond_type = np.ones(len(bonds), dtype=np.uint8)

    # Process each bond
    for bond_index in range(len(bonds)):
        bond = bonds[bond_index]
        vertex_a, vertex_b = bond

        # Find faces that contain vertices a and b
        vertex_a_faces = np.where(faces == vertex_a)[0]
        vertex_b_faces = np.where(faces == vertex_b)[0]
        common_faces = np.intersect1d(vertex_a_faces, vertex_b_faces)

        if len(common_faces) > 2:
            print(vertex_a_faces, vertex_b_faces, common_faces)
            raise IndexError("Too many common faces")

        if len(common_faces) == 2:
            # Process the two common faces
            face1, face2 = common_faces
            face_vertices1 = faces[face1]
            face_vertices2 = faces[face2]

            # Extract other vertices in the faces
            other_vertices1 = np.setdiff1d(face_vertices1, bond)
            other_vertices2 = np.setdiff1d(face_vertices2, bond)

            if len(other_vertices1) != 1 or len(other_vertices2) != 1:
                print(other_vertices1, other_vertices2)
                raise IndexError("Bond neighbour count error")

            assert vertex_a < vertex_b

            neighbour_vertex1 = other_vertices1[0]
            neighbour_vertex2 = other_vertices2[0]

            # Ensure neighbour_vertex1 < neighbour_vertex2
            if neighbour_vertex1 > neighbour_vertex2:
                neighbour_vertex1, neighbour_vertex2 = neighbour_vertex2, neighbour_vertex1

            neighbour_info.append([face1, face2, neighbour_vertex1, vertex_a, vertex_b, neighbour_vertex2])

    # Remove duplicate neighbours
    neighbour_info = np.unique(np.array(neighbour_info), axis=0)
    
    return points, faces, bonds, bond_type, neighbour_info



points,faces,bonds,bond_type,neighbour = process_mesh(mesh)


## build MD mdoel 

In [2]:
from src import agent, stl2in
import numpy as np
import os
import multiprocessing as mp


f_dir = 'spinodal_model'
box = np.array([[0,400],[0,400],[0,400]])

for root,dirs,files in os.walk(f_dir):
    for file in files:
        if file.endswith('.stl') and file[0]!='.':
            stl_file = os.path.join(root,file)
            stl2in.stl2dat(stl_file,stl_file[:-4]+'.dat',box,4,['1 0 0'],['1 0 0 0'],allow_overlap=True)

## use multiprocess for build dat file

In [1]:
import numpy as np
import os
import multiprocessing as mp
from multiprocessing import Manager
from src import agent, stl2in


f_dir = 'spinodal_model'
box = np.array([[0,400],[0,400],[0,400]])

num_process =62
mgr = Manager()
jobList = mgr.list([])

for root,dirs,files in os.walk(f_dir):
    for file in files:
        if file.endswith('.stl') and file[0]!='.':
            stl_file = os.path.join(root,file)
            jobList.append([stl_file,stl_file[:-4]+'.dat',box,4,['1 0 0'],['1 0 0 0']])

def run():
    while True:
        if len(jobList) == 0:
            break;
        job = jobList.pop()
        print(job)
        stl2in.stl2dat(*job,allow_overlap=True)


if __name__ =='__main__':

    processes = []
    for i in range(num_process):
        p = mp.Process(target=run,args=())
        processes.append(p)
        p.start()

    for p in processes:
        p.join()
        
    
    



['spinodal_model/0.50_0.10_0.00_7.stl', 'spinodal_model/0.50_0.10_0.00_7.dat', array([[  0, 400],
       [  0, 400],
       [  0, 400]]), 4, ['1 0 0'], ['1 0 0 0']]
['spinodal_model/0.20_0.10_0.60_7.stl', 'spinodal_model/0.20_0.10_0.60_7.dat', array([[  0, 400],
       [  0, 400],
       [  0, 400]]), 4, ['1 0 0'], ['1 0 0 0']]
['spinodal_model/0.20_0.60_0.10_10.stl', 'spinodal_model/0.20_0.60_0.10_10.dat', array([[  0, 400],
       [  0, 400],
       [  0, 400]]), 4, ['1 0 0'], ['1 0 0 0']]['spinodal_model/0.00_0.00_0.80_7.stl', 'spinodal_model/0.00_0.00_0.80_7.dat', array([[  0, 400],
       [  0, 400],
       [  0, 400]]), 4, ['1 0 0'], ['1 0 0 0']]

['spinodal_model/0.30_0.60_0.10_7.stl', 'spinodal_model/0.30_0.60_0.10_7.dat', array([[  0, 400],
       [  0, 400],
       [  0, 400]]), 4, ['1 0 0'], ['1 0 0 0']]
['spinodal_model/0.00_0.00_0.20_5.stl', 'spinodal_model/0.00_0.00_0.20_5.dat', array([[  0, 400],
       [  0, 400],
       [  0, 400]]), 4, ['1 0 0'], ['1 0 0 0']]['spinoda

In [ ]:
from numba import njit,jit
import numpy as np

@njit
def go():
    for x in range(1000):
        a = np.random.uniform(0,1,1000)
    
    